In [67]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.decomposition import decomposition 

data = load_boston()
x_train, x_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=.4)
y_train, y_test = y_train.reshape((-1, 1)), y_test.reshape((-1, 1))
x_train.shape, x_test.shape, y_train.shape, y_test.shape
X,y=nd.array(x_train),nd.array(y_train)

## Mxnet

In [68]:
from mxnet.gluon import nn,Trainer,loss
from mxnet import autograd,nd 

net=nn.Sequential()
net.add(nn.Dense(12,activation='relu'),
       nn.Dropout(.4),
       nn.Dense(4,activation='relu'),
       nn.Dropout(.5),
       nn.Dense(1,activation='sigmoid'))
net.initialize() 
trainer=Trainer(net.collect_params(),'adam',{'learning_rate':.01})
m=X.shape[0]
l2Loss=loss.L2Loss()
for i in range(10):
    with autograd.record():
        los=l2Loss(net(X),y)
    los.backward() 
    trainer.step(m) 
    print(f'{i}: ,{los.sum()/m}')

0: ,
[288.36652]
<NDArray 1 @cpu(0)>
1: ,
[287.734]
<NDArray 1 @cpu(0)>
2: ,
[287.11337]
<NDArray 1 @cpu(0)>
3: ,
[285.9111]
<NDArray 1 @cpu(0)>
4: ,
[284.11627]
<NDArray 1 @cpu(0)>
5: ,
[281.94537]
<NDArray 1 @cpu(0)>
6: ,
[280.7585]
<NDArray 1 @cpu(0)>
7: ,
[279.5382]
<NDArray 1 @cpu(0)>
8: ,
[278.89124]
<NDArray 1 @cpu(0)>
9: ,
[278.65057]
<NDArray 1 @cpu(0)>


## 纯手写

In [77]:
X,y=x_train,y_train
w1, b1 = np.random.uniform(size=(X.shape[1], 10)), np.zeros((1, 10))
w2, b2 = np.random.uniform(size=(10, 7)), np.zeros((1, 7))
w3, b3 = np.random.uniform(size=(7, 1)), np.zeros((1, 1))


def relu(x):
    x[x < 0] = 0
    return x


def sigmoid(x):
    return 1/(1+np.exp(-x))


params = [w1, b1, w2, b2, w3, b3]

In [78]:

m = X.shape[0]


def forward_with_Dropout(X, params, keeps=[.5, .4]):
    w1, b1, w2, b2, w3, b3 = params
    Z1 = X.dot(w1)+b1
    A1 = relu(Z1)
    D1 = np.random.uniform(0, 1, A1.shape)
    keep_1 = keeps[0]
    D1 = D1 < keep_1
    A1 = A1*D1
    A1 /= keep_1

    Z2 = A1.dot(w2)+b2
    A2 = relu(Z2)
    D2 = np.random.uniform(0, 1, A2.shape)
    keep_2 = keeps[-1]
    D2 = D2 < keep_2
    A2 = A2*D2
    A2 /= keep_2

    Z3 = A2.dot(w3)+b3
    A3 = sigmoid(Z3)
    cache = (Z1, A1, D1, Z2, A2, D2, Z3)
    return A3, cache


forward_with_Dropout(X,  params)

(array([[1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [0.9999999],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.       ],
        [1.  

In [79]:
def backward(y,A3,cache):
    Z1, A1, D1, Z2, A2, D2, Z3 = cache
    dZ3 = A3-y
    dW3 = 1/m*A2.T.dot(dZ3)
    db3 = 1/m*dZ3.sum(axis=0, keepdims=True)

    dA2 = dZ3.dot(w3.T)/keep_2
    dA2 *= D2
    dZ2 = dA2*(dA2 > 0)

    dW2 = 1/m*A1.T.dot(dZ2)
    db2 = 1/m*dZ2.sum(axis=0, keepdims=True)

    dA1 = dZ2.dot(w2.T)/keep_1
    dA1 *= D1
    dZ1 = dA1*(dA1 > 0)
    dW1 = 1/m*X.T.dot(dZ1)
    db1 = 1/m*dZ1.sum(axis=0, keepdims=True)
    grads = [dW1, db1, dW2, db2, dW3, db3]
    return grads

In [80]:
lr = .1
for i in range(100):
    A3, cache = forward_with_Dropout(X, params)
    loss = 1/(2*m)*np.square(A3-y).sum()
    if (i % 10 == 0):
        print(f'epoch:{i},loss: {loss}')
    grads = backward(y, A3, cache)
    for i in range(len(grads)):
        params[i] -= grads[i]*lr

epoch:0,loss: 277.6214687620308
epoch:10,loss: 277.08128712906654
epoch:20,loss: 277.0812871287129
epoch:30,loss: 277.0812871287129
epoch:40,loss: 277.0812871287129
epoch:50,loss: 277.0812871287129
epoch:60,loss: 277.0812871287129
epoch:70,loss: 277.0812871287129
epoch:80,loss: 277.0812871287129
epoch:90,loss: 277.0812871287129
